# Data Quality Validation

Comprehensive validation of final merged dataset after applying 200K population threshold.

## Setup

In [1]:
# Imports, settings, and load final merged dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("mode.copy_on_write", True)
sns.set_style("whitegrid")

df = pd.read_csv("../01_data/clean/final_merged_150k.csv")

# Create county-state identifier (some county names appear in multiple states)
df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]

# Identify which year column to use
year_col = "Year" if "Year" in df.columns else "year"

print("=" * 60)
print("FINAL MERGED DATASET (≥200K POPULATION)")
print("=" * 60)
print(f"\nDataset shape: {df.shape[0]:,} observations × {df.shape[1]} variables")
print(f"Time period: {df[year_col].min():.0f}-{df[year_col].max():.0f}")
print(f"Unique county-state combinations: {df['county_state'].nunique()}")
print(f"States: {df['STNAME'].nunique()}")

FINAL MERGED DATASET (≥200K POPULATION)

Dataset shape: 1,448 observations × 18 variables
Time period: 2006-2015
Unique county-state combinations: 150
States: 12


## Missing Values Analysis

Identify missing data patterns and assess impact on analysis quality.

In [2]:
# Analyze missing values and key insights
print("\n" + "=" * 60)
print("MISSING VALUES ANALYSIS")
print("=" * 60)

if "county_state" not in df.columns:
    df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame(
    {"Missing Count": missing, "Missing %": missing_pct}
).sort_values("Missing Count", ascending=False)

print("\nVariables with missing values:")
print(missing_df[missing_df["Missing Count"] > 0])

print("\n" + "-" * 60)
print("KEY INSIGHTS:")
print("-" * 60)
print(
    f"Deaths missingness: {missing_pct['Deaths']:.2f}% (CDC suppression for <10 deaths)"
)
print(
    f"ARCOS missingness: {missing_pct.get('DOSAGE_UNIT', 0):.2f}% (counties with no opioid shipments)"
)
print(f"\n✓ All counties have population data: {df['population'].notna().all()}")
print(f"✓ All observations have year data: {df[year_col].notna().all()}")


MISSING VALUES ANALYSIS

Variables with missing values:
                                 Missing Count  Missing %
merge_on_arcos                             881      60.84
TOTAL_MME                                  881      60.84
BUYER_COUNTY                               881      60.84
BUYER_STATE                                881      60.84
Deaths                                     489      33.77
Drug/Alcohol Induced Cause Code            489      33.77
Drug/Alcohol Induced Cause                 489      33.77

------------------------------------------------------------
KEY INSIGHTS:
------------------------------------------------------------
Deaths missingness: 33.77% (CDC suppression for <10 deaths)
ARCOS missingness: 0.00% (counties with no opioid shipments)

✓ All counties have population data: True
✓ All observations have year data: True


## Temporal Coverage and Panel Balance

Verify complete time series coverage (2006-2015) for all counties in the dataset.

In [3]:
# Check temporal coverage and panel balance
print("\n" + "=" * 60)
print("TEMPORAL COVERAGE")
print("=" * 60)

if "county_state" not in df.columns:
    df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]
print(f"\nYear range: {df[year_col].min():.0f} to {df[year_col].max():.0f}")
print(f"Total years: {df[year_col].nunique()}")

obs_per_year = df[year_col].value_counts().sort_index()
print("\nObservations per year:")
print(obs_per_year)

print("\n" + "-" * 60)
print("PANEL BALANCE CHECK:")
print("-" * 60)

expected_years = 10  # 2006-2015
county_year_counts = df.groupby("county_state")[year_col].count()
balanced_counties = (county_year_counts == expected_years).sum()
unbalanced_counties = (county_year_counts != expected_years).sum()

print(f"Counties with complete 10-year data: {balanced_counties}")
print(f"Counties with incomplete data: {unbalanced_counties}")

if unbalanced_counties > 0:
    print(f"\nFound {unbalanced_counties} counties with incomplete time series.")
    print("These counties crossed 200K population during 2006-2015.")
    print("\nCounties with incomplete data (first 10):")
    incomplete = county_year_counts[county_year_counts != expected_years]
    for county, count in incomplete.head(10).items():
        print(f"  - {county}: {count} years (missing {expected_years - count})")
    print(f"\nRecommendation for difference-in-differences analysis:")
    print(f"  - Use only {balanced_counties} balanced counties for clean estimates")
    print("  - OR include all counties with appropriate controls")
else:
    print("\nAll counties have complete 10-year time series")


TEMPORAL COVERAGE

Year range: 2006 to 2015
Total years: 10

Observations per year:
Year
2006    135
2007    143
2008    143
2009    143
2010    146
2011    146
2012    147
2013    148
2014    147
2015    150
Name: count, dtype: int64

------------------------------------------------------------
PANEL BALANCE CHECK:
------------------------------------------------------------
Counties with complete 10-year data: 135
Counties with incomplete data: 15

Found 15 counties with incomplete time series.
These counties crossed 200K population during 2006-2015.

Counties with incomplete data (first 10):
  - Alamance County, North Carolina: 6 years (missing 4)
  - Beaufort County, South Carolina: 9 years (missing 1)
  - DeSoto County, Mississippi: 9 years (missing 1)
  - Deschutes County, Oregon: 9 years (missing 1)
  - Dorchester County, South Carolina: 1 years (missing 9)
  - Forsyth County, Georgia: 9 years (missing 1)
  - Iredell County, North Carolina: 9 years (missing 1)
  - Johnston Coun

## Geographic Distribution

Analyze state and county coverage, treatment versus control group composition.

In [4]:
# Analyze geographic coverage and treatment/control group composition
print("\n" + "=" * 60)
print("GEOGRAPHIC COVERAGE")
print("=" * 60)

if "county_state" not in df.columns:
    df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]
print(f"\nStates: {sorted(df['STNAME'].unique())}")
print(f"Total states: {df['STNAME'].nunique()}")

counties_per_state = (
    df.groupby("STNAME")["county_state"].nunique().sort_values(ascending=False)
)
print("\nCounties per state:")
print(counties_per_state)

obs_per_state = df["STNAME"].value_counts().sort_index()
print("\nObservations per state:")
print(obs_per_state)

print("\n" + "-" * 60)
print("TREATMENT/CONTROL CLASSIFICATION:")
print("-" * 60)
print("TREATMENT STATES:")
print("  - Florida: Policy implemented 2010 (prescription limits)")
print("  - Washington: Policy implemented 2012 (prescription limits)")
print("\nCONTROL STATES:")
control_states = [
    s for s in df["STNAME"].unique() if s not in ["Florida", "Washington"]
]
for state in sorted(control_states):
    count = df[df["STNAME"] == state]["county_state"].nunique()
    print(f"  - {state} ({count} counties)")

fl_obs = df[df["STNAME"] == "Florida"].shape[0]
wa_obs = df[df["STNAME"] == "Washington"].shape[0]
control_obs = df[~df["STNAME"].isin(["Florida", "Washington"])].shape[0]

print(f"\nObservation distribution:")
print(f"  Florida (treatment): {fl_obs:,} ({fl_obs/len(df)*100:.1f}%)")
print(f"  Washington (treatment): {wa_obs:,} ({wa_obs/len(df)*100:.1f}%)")
print(f"  Control states: {control_obs:,} ({control_obs/len(df)*100:.1f}%)")


GEOGRAPHIC COVERAGE

States: ['California', 'Colorado', 'Florida', 'Georgia', 'Idaho', 'Mississippi', 'Montana', 'North Carolina', 'Oregon', 'South Carolina', 'Tennessee', 'Washington']
Total states: 12

Counties per state:
STNAME
California        33
Florida           31
North Carolina    18
Georgia           14
South Carolina    12
Colorado          10
Washington        10
Tennessee          9
Oregon             7
Mississippi        3
Idaho              2
Montana            1
Name: county_state, dtype: int64

Observations per state:
STNAME
California        324
Colorado          100
Florida           299
Georgia           130
Idaho              20
Mississippi        29
Montana             4
North Carolina    174
Oregon             69
South Carolina    110
Tennessee          89
Washington        100
Name: count, dtype: int64

------------------------------------------------------------
TREATMENT/CONTROL CLASSIFICATION:
------------------------------------------------------------
TREA

## Summary Statistics for Key Variables

Descriptive statistics for population, mortality, and opioid shipment measures.

In [5]:
# Summary statistics for population, mortality, and opioid shipment measures
print("\n" + "=" * 60)
print("KEY VARIABLE SUMMARY STATISTICS")
print("=" * 60)

# Population
print("\n1. POPULATION:")
print(df["population"].describe())
print(f"   Min: {df['population'].min():,.0f}")
print(f"   Max: {df['population'].max():,.0f}")
print(f"   ✓ All counties ≥150K threshold: {(df['population'] >= 150000).all()}")

# Deaths
print("\n2. DEATHS (Mortality):")
deaths_nonmissing = df["Deaths"].dropna()
print(
    f"   Non-missing observations: {len(deaths_nonmissing):,} ({len(deaths_nonmissing)/len(df)*100:.1f}%)"
)
print(deaths_nonmissing.describe())
print(f"   Min: {deaths_nonmissing.min():.0f}")
print(f"   Max: {deaths_nonmissing.max():.0f}")

# Check for ARCOS variables (MME, DOSAGE_UNIT, etc.)
arcos_cols = [
    col
    for col in df.columns
    if any(x in col.upper() for x in ["MME", "DOSAGE", "BUYER"])
]
if arcos_cols:
    print("\n3. ARCOS VARIABLES:")
    for col in arcos_cols[:3]:
        nonmissing = df[col].dropna()
        print(f"\n   {col}:")
        print(
            f"   Non-missing: {len(nonmissing):,} ({len(nonmissing)/len(df)*100:.1f}%)"
        )
        if pd.api.types.is_numeric_dtype(df[col]):
            print(f"   Mean: {nonmissing.mean():,.2f}")
            print(f"   Median: {nonmissing.median():,.2f}")
            print(f"   Range: [{nonmissing.min():,.2f}, {nonmissing.max():,.2f}]")

# Calculate mortality rate per 100K
if "Deaths" in df.columns:
    df["mortality_rate_per_100k"] = (df["Deaths"] / df["population"]) * 100000
    print("\n4. MORTALITY RATE (per 100,000 population):")
    mort_rate = df["mortality_rate_per_100k"].dropna()
    print(f"   Mean: {mort_rate.mean():.2f}")
    print(f"   Median: {mort_rate.median():.2f}")
    print(f"   Range: [{mort_rate.min():.2f}, {mort_rate.max():.2f}]")


KEY VARIABLE SUMMARY STATISTICS

1. POPULATION:
count    1.448000e+03
mean     5.775940e+05
std      9.403471e+05
min      1.500850e+05
25%      1.968205e+05
50%      3.076625e+05
75%      6.113122e+05
max      1.008542e+07
Name: population, dtype: float64
   Min: 150,085
   Max: 10,085,416
   ✓ All counties ≥150K threshold: True

2. DEATHS (Mortality):
   Non-missing observations: 959 (66.2%)
count    959.000000
mean      54.429614
std       56.316832
min       10.000000
25%       21.000000
50%       33.000000
75%       65.500000
max      476.000000
Name: Deaths, dtype: float64
   Min: 10
   Max: 476

3. ARCOS VARIABLES:

   BUYER_STATE:
   Non-missing: 567 (39.2%)

   BUYER_COUNTY:
   Non-missing: 567 (39.2%)

   TOTAL_MME:
   Non-missing: 567 (39.2%)
   Mean: 7,705,074,974.72
   Median: 256,802,921.99
   Range: [54,897,545.44, 4,174,281,314,231.23]

4. MORTALITY RATE (per 100,000 population):
   Mean: 12.62
   Median: 11.43
   Range: [2.59, 68.31]


## Data Integrity Validation

Check for data quality issues: duplicate observations, negative values, logical inconsistencies.

In [6]:
# Data integrity checks: negative values, threshold, year range, duplicates, logical consistency
print("\n" + "=" * 60)
print("DATA INTEGRITY CHECKS")
print("=" * 60)

if "county_state" not in df.columns:
    df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]
# Check 1: No negative values
print("\n1. NEGATIVE VALUE CHECK:")
numeric_cols = df.select_dtypes(include=[np.number]).columns
has_negatives = False
for col in numeric_cols:
    if (df[col] < 0).any():
        neg_count = (df[col] < 0).sum()
        print(f"   ⚠ {col}: {neg_count} negative values")
        has_negatives = True
if not has_negatives:
    print("   ✓ No negative values found in numeric columns")
# Check 2: Population threshold validation
print("\n2. POPULATION THRESHOLD:")
below_threshold = (df["population"] < 200000).sum()
if below_threshold > 0:
    print(f"   ⚠ WARNING: {below_threshold} observations below 200K threshold!")
else:
    print("   ✓ All observations meet ≥200K population threshold")
# Check 3: Year range validation
print("\n3. YEAR RANGE:")
expected_years = set(range(2006, 2016))
actual_years = set(df[year_col].unique())
if actual_years == expected_years:
    print("   ✓ All expected years present (2006-2015)")
else:
    missing_years = expected_years - actual_years
    extra_years = actual_years - expected_years
    if missing_years:
        print(f"   ⚠ Missing years: {missing_years}")
    if extra_years:
        print(f"   ⚠ Unexpected years: {extra_years}")
# Check 4: Duplicate observations (using county_state identifier)
print("\n4. DUPLICATE CHECK:")
id_cols = ["county_state", year_col]
duplicates = df.duplicated(subset=id_cols).sum()
if duplicates > 0:
    print(f"   ⚠ WARNING: {duplicates} duplicate county-year observations!")
    print("\n   Sample duplicate rows:")
    dup_mask = df.duplicated(subset=id_cols, keep=False)
    print(df[dup_mask][["county_state", year_col, "population", "Deaths"]].head(10))
else:
    print("   ✓ No duplicate county-year observations")
# Check 5: Deaths > Population (impossible)
print("\n5. LOGICAL CONSISTENCY:")
if "Deaths" in df.columns:
    impossible = (df["Deaths"] > df["population"]).sum()
    if impossible > 0:
        print(f"   ⚠ WARNING: {impossible} observations with deaths > population!")
    else:
        print("   ✓ No observations with deaths exceeding population")


DATA INTEGRITY CHECKS

1. NEGATIVE VALUE CHECK:
   ✓ No negative values found in numeric columns

2. POPULATION THRESHOLD:
   ⚠ WARNING: 381 observations below 200K threshold!

3. YEAR RANGE:
   ✓ All expected years present (2006-2015)

4. DUPLICATE CHECK:
   ✓ No duplicate county-year observations

5. LOGICAL CONSISTENCY:
   ✓ No observations with deaths exceeding population


## Extreme Outlier Detection

Identify and remove data errors in opioid shipment totals (values >10 billion MME indicate upstream supply chain transactions).

In [7]:
# Detect and remove extreme outliers in opioid shipment totals
print("\n" + "=" * 60)
print("EXTREME OUTLIER DETECTION")
print("=" * 60)

if "county_state" not in df.columns:
    df["county_state"] = df["CTYNAME"] + ", " + df["STNAME"]
if "TOTAL_MME" in df.columns:
    print("\nChecking TOTAL_MME for extreme outliers...")
    extreme_threshold = 1e10
    extreme_outliers = df[df["TOTAL_MME"] > extreme_threshold]
    if len(extreme_outliers) > 0:
        print(f"\n⚠ Found {len(extreme_outliers)} extreme outliers (>10 billion MME):")
        for idx, row in extreme_outliers.iterrows():
            print(
                f"   {row['county_state']}, {row[year_col]}: {row['TOTAL_MME']:,.0f} MME"
            )
        print(f"\nRemoving {len(extreme_outliers)} extreme outliers...")
        df = df[df["TOTAL_MME"] <= extreme_threshold].copy()
        df.to_csv("../01_data/clean/final_merged_150k.csv", index=False)
        print(f"✓ Cleaned dataset saved ({len(df)} observations remaining)")
    else:
        print("✓ No extreme outliers detected")


EXTREME OUTLIER DETECTION

Checking TOTAL_MME for extreme outliers...

⚠ Found 1 extreme outliers (>10 billion MME):
   Lane County, Oregon, 2010: 4,174,281,314,231 MME

Removing 1 extreme outliers...
✓ Cleaned dataset saved (566 observations remaining)
